In [ ]:
#import library yang digunakan
from tabulate import tabulate
import instagrapi
from instagrapi import Client
import json

import pandas as pd
import numpy as np
import re

from datetime import date, datetime, timedelta
import pytz
import time

timezone_utc = pytz.timezone('UTC')
timezone_indo = pytz.timezone('Asia/Jakarta')

In [ ]:
# login to instagram
client = Client()
client.login("kazuyatakeshi1", "bimoli216")    

True

### Fungsi untuk mencari data user

In [ ]:
def get_user_info(target_user):
    now = datetime.now(timezone_indo)
    like = 0
    comment = 0
    last_post = None
    last_post_interval = None
    last_story = None
    last_story_interval = None
    
    print("Scraping data from user: ", target_user)
    
    
    # get user id from username
    user_id = client.user_id_from_username(target_user)
    user_inf = client.user_info(user_id)
    
    print("Getting user data...")
    print("now: ", now)
    
    # get all media from user (tanpa delay, pasti error keblock akunnya)
    # aku ganti menjadi 10 post terakhir aja
    user_media = client.user_medias(user_id, amount=10)
    # cek if post exist
    if user_media:
        datetime_list = []
        
        # get like count, comment count, and latest datetime from all media
        for media in user_media:
            like += media.like_count
            comment += media.comment_count
            datetime_list.append(media.taken_at)
        
        latest_datetime = max(datetime_list)
        latest_index = datetime_list.index(latest_datetime)
        
        last_post = str(user_media[latest_index].taken_at)
        last_post_obj = datetime.strptime(last_post, '%Y-%m-%d %H:%M:%S%z')
        # print("last post: ", last_post_obj)
        
        # change datetime to indonesia timezone
        last_post_obj = last_post_obj.astimezone(timezone_indo)
        last_post = last_post_obj

        # count interval
        last_post_interval = (now - last_post_obj).days
        # print("last post interval: ", last_post_interval)
    
    
    # get last story from user
    # ngambil 25 stori teratas
    user_story = client.user_stories(user_id, amount=25)
    # cek if story exist
    if user_story:
        # stori terakhir yang dibuat
        user_story = user_story[-1]
        
        # get current datetime of story
        last_story = str(user_story.taken_at)
        last_story_obj = datetime.strptime(last_story, '%Y-%m-%d %H:%M:%S%z')
        # print("last story: ", last_story_obj)
        
        # change datetime to indonesia timezone
        last_story_obj = last_story_obj.astimezone(timezone_indo)
        last_story = last_story_obj
        
        # count interval
        last_story_interval = (now - last_story_obj).seconds // 3600
        # print("last story interval: ", last_story_interval)
    
    # append all data to profile_list
    profile_list = ({
            'id_user':user_inf.pk, 
            'username':user_inf.username, 
            'fullname':user_inf.full_name, 
            'is_private':user_inf.is_private,  
            'is_verified':user_inf.is_verified,  
            'follower_count': user_inf.follower_count, 
            'following_count': user_inf.following_count,
            'media_count':user_inf.media_count,
            'like_count(last10post)': like,
            'comment_count(last10post)': comment,
            'last_post': str(last_post),
            'last_post_interval(days)': last_post_interval,
            'last_story': str(last_story),
            'last_story_interval(hours)': last_story_interval,
            })
    
    print("Scraping Finish!!!")
    return profile_list

### Fungsi untuk mencari data feed user

In [ ]:
def get_feed_info(target_user):
    data_feed_IG = []
    
    now = datetime.now(timezone_indo)
    
    print("Scraping data from user: ", target_user)
    
    #Menggambil informasi dari user
    user_id = client.user_id_from_username(target_user)
    user_inf = client.user_info(user_id)
    
    print("Getting feed data...")
    print("now: ", now)
     
    #Mengambil data dari feed user
    user_media = client.user_medias(user_inf.pk, amount=10)
    for i in range (len(user_media)):
        feed_IG = ({
                'id_user':user_inf.pk, 
                'id_media': user_media[i].pk,
                'username':user_inf.username, 
                'fullname':user_inf.full_name, 
                'taken_at':str(user_media[i].taken_at), 
                'media_type':user_media[i].media_type, 
                'comment_count':user_media[i].comment_count, 
                'like':user_media[i].like_count, 
                'caption': user_media[i].caption_text})
        data_feed_IG.append(feed_IG)
    print("Scraping Finish!!!")
    return data_feed_IG    

### Fungsi untuk mendapatkan data comment feed

In [ ]:
def get_comment_feed(target_user):
    #variable untuk nampung data info users
    data_comment_feed = []
    
    now = datetime.now(timezone_indo)
    
    print("Scraping data from user: ", target_user)
    
    user_id = client.user_id_from_username(target_user)
    user_inf = client.user_info(user_id)
    
    print("Getting feed data...")
    print("now: ", now)
    
    user_media = client.user_medias(user_inf.pk, amount=5)
        
    for media in (user_media):
        media_comments = client.media_comments(media.pk, amount=50)
        for comment in (media_comments):
                
                data_comment_feed.append({
                    'id_user':user_inf.pk, 
                    'id_media': media.pk,
                    'posting_taken_at':str(media.taken_at),
                    'id_comment': comment.pk,
                    'comment__taken_at':str(comment.created_at_utc),
                    'username_user_post':user_inf.username, 
                    'fullname_user_post':user_inf.full_name,  
                    'media_type_postingan':media.media_type,  
                    'like_postingan': media.like_count, 
                    'caption': media.caption_text,
                    'username_comment': comment.user.username,
                    'comment_text': comment.text})
    print("Scraping Finish!!!")
    return data_comment_feed

In [ ]:
'''
row = [['ID User',users_profile[0]['id_user']], 
       ['Username', users_profile[0]['username']], 
       ['Fullname',users_profile[0]['fullname']], 
       ['Private',users_profile[0]['is_private']], 
       ['Verified',users_profile[0]['is_verified']], 
       ['Follower',users_profile[0]['follower_count']], 
       ['Following',users_profile[0]['following_count']], 
       ['Media(Count)',users_profile[0]['media_count']], 
       ['Like(Count)',users_profile[0]['like_count(last10post)']], 
       ['Comment(Count)',users_profile[0]['comment_count(last10post)']], 
       ['LastPost',users_profile[0]['last_post']], 
       ['PostInterval(Days)',users_profile[0]['last_post_interval(days)']], 
       ['LastStory',users_profile[0]['last_story']], 
       ['IntervalStory(Hours)',users_profile[0]['last_story_interval(hours)']]]
header = ['Parameter', 'Value']
print(tabulate(row, header, tablefmt="github"))'''

| Parameter            | Value                                 |
|----------------------|---------------------------------------|
| ID User              | 2286735082                            |
| Username             | ranggasaputra4000                     |
| Fullname             | Rangga Saputra~Data Science & Analyst |
| Private              | False                                 |
| Verified             | False                                 |
| Follower             | 173                                   |
| Following            | 306                                   |
| Media(Count)         | 8                                     |
| Like(Count)          | 89                                    |
| Comment(Count)       | 6                                     |
| LastPost             | 2023-01-22 10:12:10+07:00             |
| PostInterval(Days)   | 51                                    |
| LastStory            |                                       |
| IntervalStory(Hours) | 